In [1]:
import os
os.chdir("..")

In [2]:
os.listdir()

['pyproject.toml',
 'README.md',
 '.gitignore',
 '.venv',
 'poetry.lock',
 '.git',
 'data',
 'notebooks',
 'reports',
 'src']

In [3]:
from src.config import PROCESSED_DATA_DIR
import pandas as pd

In [4]:
top_16_hotspots_df = pd.read_csv(PROCESSED_DATA_DIR / "top_16_hotspots.csv")
top_16_hotspots_df.head()

,Unnamed: 0,CFSAUID,PRUID,PRNAME,geometry,Latitude,Longitude,centroid,low_income_score
0,5,M2N,35,Ontario,POLYGON ((-79.40328008756583 43.75683978645180...,43.767318,-79.410939,POINT (-79.4109392512375 43.7673176050981),0.090482
1,69,M4Y,35,Ontario,POLYGON ((-79.37496923366446 43.66780414934715...,43.666578,-79.381093,POINT (-79.38109268117113 43.666578423536144),0.079082
2,75,M5A,35,Ontario,POLYGON ((-79.35582500066013 43.66322724858036...,43.656277,-79.362270,POINT (-79.36226982632692 43.65627679308786),0.078342
3,76,M5B,35,Ontario,POLYGON ((-79.37806890079342 43.66233100368292...,43.657921,-79.377941,POINT (-79.37794125331362 43.65792054324893),0.070537
4,18,M4H,35,Ontario,"POLYGON ((-79.3379155022292 43.70522552336265,...",43.704100,-79.343631,POINT (-79.34363111438358 43.70410011814641),0.070389


In [5]:
top_16_hotspots_df['low_income_score'].sum()

np.float64(0.9999999999999994)

In [6]:
relevant_data_df = top_16_hotspots_df[['CFSAUID', 'low_income_score']]
relevant_data_df

,CFSAUID,low_income_score
0,M2N,0.090482
1,M4Y,0.079082
2,M5A,0.078342
3,M5B,0.070537
4,M4H,0.070389
5,M3N,0.061238
6,M5T,0.060024
7,M4X,0.060009
8,M5G,0.057228
9,M2J,0.056017


In [7]:
relevant_data_df = relevant_data_df.rename(columns={"low_income_score" : "ratio_for_demand_calc"})
relevant_data_df
relevant_data_df.to_csv(PROCESSED_DATA_DIR / "hotspot_ratio_demand.csv")

In [8]:
demand_per_month_df = pd.read_csv(PROCESSED_DATA_DIR / "demand_per_month_by_gender.csv")
demand_per_month_df

,month,true_total,ratio_male,ratio_female,gender_male_v2,gender_female_v2
0,1,8051,0.679913,0.320087,5474,2577
1,2,8086,0.678548,0.321452,5487,2599
2,3,7582,0.678741,0.321259,5146,2436
3,4,7595,0.675198,0.324802,5128,2467
4,5,7197,0.670287,0.329713,4824,2373
5,6,7039,0.667012,0.332988,4695,2344
6,7,6969,0.661351,0.338649,4609,2360
7,8,6946,0.661402,0.338598,4594,2352
8,9,6774,0.662546,0.337454,4488,2286
9,10,6765,0.659390,0.340610,4461,2304


In [9]:
import pandas as pd

# Step 1: Cross join months with hotspots
df = demand_per_month_df.assign(key=1).merge(
    relevant_data_df.assign(key=1),
    on="key",
    how="outer"
).drop(columns="key")

# Step 2: Compute per-month per-hotspot gender counts
df["male_from_hotspot"] = (
    df["gender_male_v2"] * df["ratio_for_demand_calc"]
)

df["female_from_hotspot"] = (
    df["gender_female_v2"] * df["ratio_for_demand_calc"]
)

# Step 3: Round to whole people
df["male_from_hotspot"] = df["male_from_hotspot"].round().astype(int)
df["female_from_hotspot"] = df["female_from_hotspot"].round().astype(int)

# Final dataframe
df = df[[
    "month",
    "CFSAUID",
    "ratio_for_demand_calc",
    "gender_male_v2",
    "gender_female_v2",
    "male_from_hotspot",
    "female_from_hotspot"
]]

df.head(20)

,month,CFSAUID,ratio_for_demand_calc,gender_male_v2,gender_female_v2,male_from_hotspot,female_from_hotspot
0,1,M2N,0.090482,5474,2577,495,233
1,1,M4Y,0.079082,5474,2577,433,204
2,1,M5A,0.078342,5474,2577,429,202
3,1,M5B,0.070537,5474,2577,386,182
4,1,M4H,0.070389,5474,2577,385,181
5,1,M3N,0.061238,5474,2577,335,158
6,1,M5T,0.060024,5474,2577,329,155
7,1,M4X,0.060009,5474,2577,328,155
8,1,M5G,0.057228,5474,2577,313,147
9,1,M2J,0.056017,5474,2577,307,144


In [10]:
df.to_csv(PROCESSED_DATA_DIR / "demand_per_hotspot.csv")

In [11]:
capacities_df = pd.read_csv(PROCESSED_DATA_DIR / "ByProgramNameCapacities.csv")
capacities_df

,Unnamed: 0,PROGRAM_ID,LOCATION_NAME,CAPACITY_TYPE,CAPACITY_ACTUAL_ROOM,CAPACITY_ACTUAL_BED
0,0,11794,Na-Me-Res,Bed Based Capacity,NaN,54.0
1,1,11798,TSSS Womens' Residence,Bed Based Capacity,NaN,87.0
2,2,11812,TSSS Seaton House,Bed Based Capacity,NaN,66.0
3,3,11815,TSSS Downsview Dells,Bed Based Capacity,NaN,18.0
4,4,11831,TSSS Fort York Residence,Bed Based Capacity,NaN,68.0
5,5,11871,Sagatay,Bed Based Capacity,NaN,17.0
6,6,11891,The Scott Mission - Kensington location,Bed Based Capacity,NaN,71.0
7,7,11897,Dixon Hall Schoolhouse,Bed Based Capacity,NaN,30.0
8,8,11898,Good Shepherd Centre Barrett House,Bed Based Capacity,NaN,5.0
9,9,11899,Good Shepherd Centre,Bed Based Capacity,NaN,25.0


In [12]:
# Extract LOCATION_NAME and create CAPACITY
capacities_df["CAPACITY"] = capacities_df["CAPACITY_ACTUAL_ROOM"].fillna(0) + capacities_df["CAPACITY_ACTUAL_BED"].fillna(0)

capacities_new_df = capacities_df[["LOCATION_NAME", "CAPACITY"]]
capacities_new_df = capacities_new_df.rename(columns={"LOCATION_NAME": "SHELTER_NAME"})
capacities_new_df

,SHELTER_NAME,CAPACITY
0,Na-Me-Res,54.0
1,TSSS Womens' Residence,87.0
2,TSSS Seaton House,66.0
3,TSSS Downsview Dells,18.0
4,TSSS Fort York Residence,68.0
5,Sagatay,17.0
6,The Scott Mission - Kensington location,71.0
7,Dixon Hall Schoolhouse,30.0
8,Good Shepherd Centre Barrett House,5.0
9,Good Shepherd Centre,25.0


In [13]:
capacities_new_df.to_csv(PROCESSED_DATA_DIR / "capacities_per_shelter.csv")

In [15]:
eligibility_df = capacities_new_df[['SHELTER_NAME']]

men_eligible = pd.read_csv(PROCESSED_DATA_DIR / "eligibility_matrix_men_initial.csv")
women_eligible = pd.read_csv(PROCESSED_DATA_DIR / "eligibility_matrix_women_initial.csv")

# Rename columns
men_eligible = men_eligible.add_prefix("men_eligible_")
women_eligible = women_eligible.add_prefix("women_eligible_")

# Concatenate
eligibility_df = pd.concat(
    [eligibility_df, men_eligible, women_eligible],
    axis=1
)

eligibility_df
eligibility_df.to_csv(PROCESSED_DATA_DIR / "eligibility_df.csv")